# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True
                                  rotation_range=20,           # Rotate the image by up to 20 degrees
    width_shift_range=0.2,       # Shift the image horizontally by up to 20% of the image width
    height_shift_range=0.2,      # Shift the image vertically by up to 20% of the image height
    shear_range=0.2,             # Shear the image by up to 20 degrees
    zoom_range=0.2,              # Zoom in or out on the image by up to 20%
    horizontal_flip=True,        # Flip the image horizontally
    fill_mode='nearest'
                                  
                                  )


training_set = train_datagen.flow_from_directory(r'trainset\ulcers',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 13 images belonging to 3 classes.


### Preprocessing the Test set

In [4]:
# test_datagen = ImageDataGenerator(rescale = 1./255)
# test_set = test_datagen.flow_from_directory(r'testset\rashes',
#                                             target_size = (64, 64),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')

## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()



### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
import os

dir_path = r'C:\Users\Geetha\Desktop\std_severity\trainset\ulcers'

# Get a list of all the files in the directory
file_list = os.listdir(dir_path)

# Loop through the files and delete any with the ".ini" extension
for file_name in file_list:
    if file_name.endswith('.ini'):
        file_path = os.path.join(dir_path, file_name)
        os.remove(file_path)
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\muscle_wasting\desktop.ini')
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\yellowing_skin\desktop.ini')


In [14]:
import os
import cv2
bad_list=[]
dir=r'C:\Users\Geetha\Desktop\std_severity\trainset\ulcers'
subdir_list=os.listdir(dir)
for d in subdir_list:  
    dpath=os.path.join (dir, d) 
    if d in ['testset', 'trainset']:
        class_list=os.listdir(dpath) 
       # print (class_list)
        for klass in class_list: 
            class_path=os.path.join(dpath, klass) 
            #print(class_path)
            file_list=os.listdir(class_path) 
            for f in file_list: 
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') 
                ext=f[index+1:] 
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)

[]


In [15]:
cnn.fit(x = training_set, epochs = 25)


Epoch 1/25
1/1 [==============================] - 1s 1s/step - loss: 1.1009 - accuracy: 0.3077
Epoch 2/25
1/1 [==============================] - 0s 220ms/step - loss: 1.1779 - accuracy: 0.3846
Epoch 3/25
1/1 [==============================] - 0s 230ms/step - loss: 1.1765 - accuracy: 0.3846
Epoch 4/25
1/1 [==============================] - 0s 267ms/step - loss: 1.1046 - accuracy: 0.5385
Epoch 5/25
1/1 [==============================] - 0s 283ms/step - loss: 1.0537 - accuracy: 0.4615
Epoch 6/25
1/1 [==============================] - 0s 241ms/step - loss: 1.0509 - accuracy: 0.5385
Epoch 7/25
1/1 [==============================] - 0s 218ms/step - loss: 1.0311 - accuracy: 0.6154
Epoch 8/25
1/1 [==============================] - 0s 175ms/step - loss: 1.0123 - accuracy: 0.5385
Epoch 9/25
1/1 [==============================] - 0s 250ms/step - loss: 1.0022 - accuracy: 0.5385
Epoch 10/25
1/1 [==============================] - 0s 220ms/step - loss: 0.9720 - accuracy: 0.6154
Epoch 11/25
1/1 [=====

In [16]:

class_indices = training_set.class_indices
print(class_indices)

# Get the class label for a particular class




{'mild': 0, 'moderate': 1, 'severe': 2}


## Part 4 - Making a single prediction

In [17]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('single_prediction/ulcers_moderate.jpg', target_size = (64, 64))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)
training_set.class_indices
if result[0][0] >result[0][1] and result[0][0] >result[0][2]:
  prediction = 'mild'
elif result[0][1] >result[0][0] and result[0][1] >result[0][2]:
  prediction = 'moderate'
elif result[0][2] >result[0][0] and result[0][2] >result[0][1]:
  prediction = 'very severe'



probs = cnn.predict(test_image)
print(probs)

prob_sum = np.sum(probs)
print(prob_sum)

indices=np.argsort(probs)
print(indices)
print(probs[0][indices])
if probs[0][indices[0][0]]> probs[0][indices[0][1]]:
    print("a is larger than b")
else:
    print("b is larger than a")
    
top_3 = np.argsort(probs,axis=1)[:, -3:]
print(top_3)
class_probs = np.take_along_axis(probs, top_3, axis=1)
print(class_probs)

print(f"The test sample {test_image} ")
print("Top three Predicted classes and their corresponding probabilties\n")
for j in range(3):
        class_idx = top_3[0][2-j]
        class_prob = class_probs[0][2-j]
        print(f"\tClass {class_idx}: {class_prob:.5f}")
# print(f"Actual class to which the test sample belongs to {}  ")�


1/1 [==============================] - 0s 172ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
[[1. 0. 0.]]
1.0
[[1 2 0]]
[[0. 0. 1.]]
b is larger than a
[[1 2 0]]
[[0. 0. 1.]]
The test sample [[[[219. 169. 162.]
   [229. 179. 182.]
   [230. 184. 187.]
   ...
   [194. 138. 125.]
   [192. 140. 126.]
   [189. 140. 125.]]

  [[223. 173. 164.]
   [233. 183. 186.]
   [235. 189. 191.]
   ...
   [192. 139. 125.]
   [192. 140. 126.]
   [188. 139. 124.]]

  [[227. 177. 166.]
   [237. 187. 186.]
   [238. 192. 192.]
   ...
   [191. 139. 125.]
   [190. 141. 126.]
   [188. 139. 124.]]

  ...

  [[230. 115. 128.]
   [232. 120. 132.]
   [227. 120. 130.]
   ...
   [ 70.  43.  32.]
   [ 53.  35.  25.]
   [ 35.  26.  21.]]

  [[226. 114. 126.]
   [231. 116. 129.]
   [232. 114. 128.]
   ...
   [ 53.  36.  28.]
   [ 36.  27.  22.]
   [ 19.  13.  13.]]

  [[227. 115. 127.]
   [225. 110. 123.]
   [227. 109. 123.]
   ...
   [ 40.  26.  17.]
   [ 24.  17.  11.]
   [  8.   2.   2.]]]] 
T

In [18]:
print(prediction)

mild


In [19]:
print(result)

[[1. 0. 0.]]
